GA: Sudoku

In [5]:
import numpy as np
import pandas as pd
import copy
import random

In [6]:
class GeneticAlgorithmSudoku:
    def __init__(self, archivo, popsize=300, NGenerations=200, probCrossover=0.7, probMutation=0.4):
        # Lee archivo CSV con el sudoku y lo guardamos como matriz
        sudoku_df = pd.read_csv(archivo, header=None)
        self.sudoku_matrix = sudoku_df.values

        self.popsize = popsize
        self.NGenerations = NGenerations
        self.population = [None] * popsize
        self.fitness = np.zeros(self.popsize, int)
        self.probCrossover = probCrossover
        self.probMutation = probMutation
        self.elite = None
        self.eliteFit = None
    
    def init_population(self):
       
        allValues = set(range(1, 10))  # Conjunto de valores del 1 al 9

        # Determina los valores faltantes en cada fila del sudoku
        missingValues = [list(allValues - set(self.sudoku_matrix[i])) for i in range(9)]
        
        # Crea cada individuo de la población inicial
        for i in range(self.popsize):
            newInd = copy.deepcopy(missingValues)
            for j in range(9):
                random.shuffle(newInd[j])  # Mezcla los valores faltantes en cada fila
            self.population[i] = newInd
            self.fitness[i] = self.fitness_individual(newInd)  # Calcula la aptitud del individuo
          
    def tournament_selection(self):
        #Selecciona un individuo utilizando selección por torneo.
        i1 = np.random.randint(self.popsize)
        i2 = np.random.randint(self.popsize)
        return i1 if self.fitness[i1] <= self.fitness[i2] else i2

    def crossover(self, parent1, parent2):
        #Realiza la cruza de dos padres para crear un hijo.
        offspring = [None] * 9
        offspring[0:4] = copy.deepcopy(parent1[0:4])
        offspring[4:9] = copy.deepcopy(parent2[4:9])
        return offspring
    
    def mutation(self, individual):
        #Realiza mutación en un individuo.
        row = np.random.randint(9)
        i1, i2 = np.random.choice(len(individual[row]), size=2, replace=False)
        individual[row][i1], individual[row][i2] = individual[row][i2], individual[row][i1]
        return individual

    def fitness_individual(self, individual):
        #Calcula la aptitud de un individuo en función del número de errores.
        errors = 0
        board = self.create_board(individual)

        # Cuenta los errores en columnas (números repetidos)
        for i in range(9):
            column = [board[j][i] for j in range(9)]
            errors += 9 - len(set(column))
        
        # Cuenta los errores en las subcuadrículas de 3x3
        for i in range(0, 9, 3):
            for j in range(0, 9, 3):
                subgrid = [board[x][y] for x in range(i, i+3) for y in range(j, j+3)]
                errors += 9 - len(set(subgrid))
        return errors
            
    def create_board(self, individual):
        #Crea un tablero de sudoku completo a partir de un individuo.
        board = copy.deepcopy(self.sudoku_matrix)
        for i in range(9):
            index = 0
            for j in range(9):
                if board[i][j] == 0:
                    board[i][j] = individual[i][index]
                    index += 1
        return board
    
    def update_elite(self):
        #Actualiza el individuo élite, el mejor de la población.
        i = np.argmin(self.fitness)
        if self.eliteFit is None or self.fitness[i] < self.eliteFit:
            self.elite = copy.deepcopy(self.population[i])
            self.eliteFit = self.fitness[i]
        else:
            i = np.random.randint(self.popsize)
            self.population[i] = self.elite
            self.fitness[i] = self.eliteFit

    def exec(self):
        #Ejecutar
        self.init_population()
        self.update_elite()

        for generation in range(1, self.NGenerations + 1):
            if self.eliteFit == 0:
                break

            offspring = [None] * self.popsize
            offspring_fitness = np.zeros(self.popsize, int)

            for i in range(self.popsize):
                parent1 = self.tournament_selection()
                parent2 = self.tournament_selection()
                
                if np.random.rand() <= self.probCrossover:
                    child = self.crossover(self.population[parent1], self.population[parent2])
                else:
                    child = copy.deepcopy(self.population[parent1])
                
                if np.random.rand() <= self.probMutation:
                    child = self.mutation(child)
                
                offspring[i] = child
                offspring_fitness[i] = self.fitness_individual(child)
            
            self.population = offspring
            self.fitness = offspring_fitness
            self.update_elite()

            print(f'Generation {generation}, Fitness: {self.eliteFit}')
        
        # Imprime el mejor tablero encontrado
        print(self.create_board(self.elite))

# Ejemplo de ejecución
ga = GeneticAlgorithmSudoku('sudoku2.csv')
ga.exec()


Generation 1, Fitness: 27
Generation 2, Fitness: 25
Generation 3, Fitness: 22
Generation 4, Fitness: 22
Generation 5, Fitness: 19
Generation 6, Fitness: 19
Generation 7, Fitness: 16
Generation 8, Fitness: 15
Generation 9, Fitness: 15
Generation 10, Fitness: 15
Generation 11, Fitness: 15
Generation 12, Fitness: 15
Generation 13, Fitness: 15
Generation 14, Fitness: 14
Generation 15, Fitness: 14
Generation 16, Fitness: 14
Generation 17, Fitness: 14
Generation 18, Fitness: 13
Generation 19, Fitness: 13
Generation 20, Fitness: 12
Generation 21, Fitness: 12
Generation 22, Fitness: 12
Generation 23, Fitness: 10
Generation 24, Fitness: 10
Generation 25, Fitness: 10
Generation 26, Fitness: 10
Generation 27, Fitness: 10
Generation 28, Fitness: 10
Generation 29, Fitness: 10
Generation 30, Fitness: 10
Generation 31, Fitness: 10
Generation 32, Fitness: 10
Generation 33, Fitness: 9
Generation 34, Fitness: 9
Generation 35, Fitness: 9
Generation 36, Fitness: 9
Generation 37, Fitness: 9
Generation 38, 